In [1]:
import sys
sys.path.append('../input/U-2-Net/')
sys.path.append('../')

In [2]:
# from u2net_test import extract
# extract('../input/petfinder-pawpularity-score/train', '../input/petfinder-pawpularity-score/train_U2NET')
# extract('../input/petfinder-pawpularity-score/test', '../input/petfinder-pawpularity-score/test_U2NET')

In [3]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import IncrementalPCA
from tqdm.notebook import tqdm
import torch.nn.functional as F
from albumentations.pytorch.transforms import ToTensorV2
import seaborn as sns
import PIL.Image as Image
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import time
import pandas_profiling as pdp
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from utils.util import *
from utils.losses import *
import gc
import torch.nn as nn
import transformers as T
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import ttach as tta
import torch.nn as nn
from sklearn import metrics
import torch
from tqdm import tqdm
import math
import albumentations
import tez
import torch.optim as optim
import warnings
warnings.simplefilter('ignore')

In [4]:
class CONFIG:
    DATA_PATH = Path('../input/petfinder-pawpularity-score')
    OUTPUT_DIR = Path('../output/32')
    TEACHER_MODEL_PATH = Path('../output/21')
    TEACHER_MODEL_NAME = 'swin_large_patch4_window12_384'
    STUDENT_MODEL_NAME = 'swin_large_patch4_window7_224'
    
    batch_size = 16
    image_size = 224
    fold = 5
    epoch = 20
    lr = 1e-5
    device='cuda'
    training_step=True
    pretrained=True
    SEED=999
    MIX_UP = True
    MASK = False
if not os.path.isdir(CONFIG.OUTPUT_DIR):
    os.makedirs(CONFIG.OUTPUT_DIR)
LOGGER = init_logger(OUTPUT_DIR=CONFIG.OUTPUT_DIR)
fix_seed(CONFIG.SEED)
pet_category = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair',
 'Egyptian_Mau' ,'Maine_Coon', 'Persian', 'Ragdoll', 'Russian_Blue' ,'Siamese',
 'Sphynx', 'american_bulldog' ,'american_pit_bull_terrier', 'basset_hound',
 'beagle', 'boxer' ,'chihuahua', 'english_cocker_spaniel', 'english_setter',
 'german_shorthaired' ,'great_pyrenees', 'havanese', 'japanese_chin',
 'keeshond', 'leonberger', 'miniature_pinscher', 'newfoundland', 'pomeranian',
 'pug' ,'saint_bernard' ,'samoyed' ,'scottish_terrier', 'shiba_inu',
 'staffordshire_bull_terrier' ,'wheaten_terrier' ,'yorkshire_terrier']

In [5]:
train_df = pd.read_csv(CONFIG.DATA_PATH / 'train.csv')
train_df['path'] = train_df['Id'].map(lambda x:str(CONFIG.DATA_PATH/'train'/x)+'.jpg')
train_df['mask_path'] = train_df['Id'].map(lambda x:str(CONFIG.DATA_PATH/'train_U2NET'/x)+'.jpg')
train_df['image_size'] = train_df['path'].apply(lambda image_id : Image.open(image_id).size)
train_df['width'] = train_df['image_size'].apply(lambda x: x[0])
train_df['height'] = train_df['image_size'].apply(lambda x: x[1])

test_df = pd.read_csv(CONFIG.DATA_PATH / 'test.csv')
test_df['path'] = test_df['Id'].map(lambda x:str(CONFIG.DATA_PATH/'test'/x)+'.jpg')
test_df['image_size'] = test_df['path'].apply(lambda image_id : Image.open(image_id).size)
test_df['width'] = test_df['image_size'].apply(lambda x: x[0])
test_df['height'] = test_df['image_size'].apply(lambda x: x[1])

if CONFIG.MASK:
    train_df['mask_path'] = train_df['Id'].map(lambda x:str(CONFIG.DATA_PATH/'train_U2NET'/x)+'.jpg')
    test_df['mask_path'] = test_df['Id'].map(lambda x:str(CONFIG.DATA_PATH/'test_U2NET'/x)+'.jpg')
    
train_df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,path,mask_path,image_size,width,height
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,../input/petfinder-pawpularity-score/train/000...,../input/petfinder-pawpularity-score/train_U2N...,"(405, 720)",405,720
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,../input/petfinder-pawpularity-score/train/000...,../input/petfinder-pawpularity-score/train_U2N...,"(1032, 774)",1032,774
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,../input/petfinder-pawpularity-score/train/001...,../input/petfinder-pawpularity-score/train_U2N...,"(720, 960)",720,960
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,../input/petfinder-pawpularity-score/train/001...,../input/petfinder-pawpularity-score/train_U2N...,"(405, 720)",405,720
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,../input/petfinder-pawpularity-score/train/001...,../input/petfinder-pawpularity-score/train_U2N...,"(540, 960)",540,960


In [6]:
num_bins = int(np.floor(1+(3.3)*(np.log2(len(train_df)))))
train_df = get_train_data(train_df, train_df['Pawpularity'], n_splits = CONFIG.fold, regression=True, num_bins=num_bins)

In [7]:
train_aug = albumentations.Compose(
    [
    albumentations.Resize(CONFIG.image_size, CONFIG.image_size, p=1),
    albumentations.HueSaturationValue(
            hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
        ),
    albumentations.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
        ),
    albumentations.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=255.0,
        p=1.0,)],p=1.0,
)

teacher_aug = albumentations.Compose(
    [
        albumentations.Resize(384,384, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

test_aug = albumentations.Compose(
    [
        albumentations.Resize(CONFIG.image_size, CONFIG.image_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

def mixup(x: torch.Tensor, y: torch.Tensor, alpha: float = 1.0):
    assert alpha > 0, "alpha should be larger than 0"
    assert x.size(0) > 1, "Mixup cannot be applied to a single instance."

    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(x.size()[0])
    mixed_x = lam * x + (1 - lam) * x[rand_index, :]
    target_a, target_b = y, y[rand_index]
    return mixed_x, target_a, target_b, lam

In [8]:
class PawpularDataset:
    def __init__(self, df, targets, augmentations, mask=True):
        self.image_paths = df['path'].tolist()
        self.mask = mask
        if self.mask:
            self.mask_paths = df['mask_path'].tolist()
        self.targets = targets
        if self.targets is None:
            self.targets = torch.ones(len(self.image_paths))
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.mask:
            mask = cv2.imread(self.mask_paths[item])
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
            idx = np.where(mask > 127.5)
            h_max = idx[0].min()
            h_min = idx[0].max()
            w_max = idx[1].min()
            w_min = idx[1].max()
        
            image = image[h_max:h_min, w_max:w_min,:]
            mask = mask[h_max:h_min, w_max:w_min]
            image = image * np.expand_dims(mask > 127.5, 2)
            
            if self.augmentations is not None:
                augmented = self.augmentations(image=image)
                image = augmented["image"]
        else:
            if self.augmentations is not None:
                augmented = self.augmentations(image=image)
                image = augmented["image"]

        targets = self.targets[item]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float)
        }


In [9]:
class PawpularModel(nn.Module):
    def __init__(self, pet_classify_model, model_name):
        super().__init__()
        self.pet_classify_model = pet_classify_model
        self.pet_classify_model.requires_grad = False
        self.model = timm.create_model(model_name, pretrained=CONFIG.pretrained, in_chans=3)
#         self.model.patch_embed.proj=nn.Conv2d(4, 96, kernel_size=(4, 4), stride=(4, 4))
        self.model.head = nn.Linear(self.model.head.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(128+37, 64)
        self.dense2 = nn.Linear(64, 1)

    def forward(self, image):
        p = self.pet_classify_model(F.adaptive_avg_pool2d(image, (224,224)))
        p = torch.softmax(p, dim=1)
        x = self.model(image)
        x = self.dropout(x)
        x = torch.cat([x, p], dim=1)
        x = self.dense1(x)
        x = self.dense2(x)
        return torch.sigmoid(x.squeeze(1))
    
class pet_categor_extract_model(nn.Module):
    def __init__(self,class_num):
        super().__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=False, in_chans=3)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense = nn.Linear(128
                               , class_num)

    def forward(self, image):
        x = self.model(image)
        x = self.dropout(x)
        x = self.dense(x)
        return x.squeeze(1)

In [10]:
def train_fn(train_loader, model, criterion, optimizer, scheduler, batch_size, epoch, device):
    start = end = time.time()
    losses = AverageMeter()
    model.train()
    for iter, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()
        img ,target = data['image'], data['targets']
        img = img.to(device)
        target = target.to(device)
        if torch.rand(1)[0] < 0.5 and CONFIG.MIX_UP:
            mix_images, target_a, target_b, lam = mixup(img, target, alpha=0.5)
            y_preds = model(mix_images)
            loss = criterion(y_preds, target_a) * lam + \
                (1 - lam) * criterion(y_preds, target_b)
        else:
            y_preds = model(img)
            loss = criterion(y_preds, target)
        # record loss
        losses.update(loss.item(), batch_size)
        loss.backward()
        optimizer.step()
    scheduler.step()
    return losses.avg

def valid_fn(valid_loader, model, criterion, device):
    start = end = time.time()
    losses = AverageMeter()

    # switch to evaluation mode
    model.eval()
    preds = []

    for iter, data in enumerate(valid_loader):
        img ,target = data['image'], data['targets']
        img = img.to(device)
        target = target.to(device)
        batch_size = target.size(0)

        # compute loss
        with torch.no_grad():
            y_preds = model(img)

        loss = criterion(y_preds, target)
        losses.update(loss.item(), batch_size)

        # record score
        preds.append(y_preds.to("cpu").numpy())

    predictions = np.concatenate(preds)
    return losses.avg, predictions

def give_label(train_loader, teacher_model, device='cuda'):
    start = end = time.time()

    # switch to evaluation mode
    teacher_model.eval()
    preds = []

    for iter, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        img = data['image']
        img = img.to(device)

        # compute loss
        with torch.no_grad():
            y_preds = teacher_model(img)

        # record score
        preds.append(y_preds.to("cpu").numpy())

    soft_target = np.concatenate(preds) * 100
    return soft_target
    

In [11]:
def train_loop(train, fold_):
    LOGGER.info(f"========== fold: {fold_} training ==========")

    # ====================================================
    # Data Loader
    # ====================================================
    cl_model = pet_categor_extract_model(class_num=37)
    cl_model.to(CONFIG.device)
    
    tc_model =  PawpularModel(cl_model, model_name=CONFIG.TEACHER_MODEL_NAME)
    tc_model.to(CONFIG.device)
    tc_model.load_state_dict(fix_model_state_dict(torch.load(CONFIG.TEACHER_MODEL_PATH / f"{CONFIG.TEACHER_MODEL_NAME}_{fold_}_best.pth")["model"]))
    
    if torch.cuda.device_count()>1:
        tc_model=nn.DataParallel(tc_model)
    
    st_model =  PawpularModel(cl_model, model_name=CONFIG.STUDENT_MODEL_NAME)
    st_model.to(CONFIG.device)           
    if torch.cuda.device_count()>1:
        st_model=nn.DataParallel(st_model)
    earlystopping= EarlyStopping(patience=3, path=CONFIG.OUTPUT_DIR / f"{CONFIG.STUDENT_MODEL_NAME}_{fold_}_latest.pth")
    
    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    train_idx = train[train.fold!=fold_].index
    val_idx = train[train.fold ==fold_].index
    train_folds = train.loc[train_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)
    
    teacher_dataset = PawpularDataset(
        train_folds, targets=None,
        augmentations=teacher_aug, mask=CONFIG.MASK
    )
    
    val_dataset = PawpularDataset(
        valid_folds, targets=valid_folds['Pawpularity']/100,
        augmentations=test_aug, mask=CONFIG.MASK
    )
    teacher_loader = DataLoader(
        teacher_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=False,
        pin_memory=True,
        num_workers=4,
        drop_last=False,
    )
    
    valid_loader = DataLoader(
        val_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=False,
        pin_memory=True,
        num_workers=4,
        drop_last=False,
    )

    soft_target = give_label(teacher_loader, tc_model, CONFIG.device)
    tc_model = None
    gc.collect()
    
    train_dataset = PawpularDataset(
        train_folds, targets=soft_target/100,
        augmentations=train_aug, mask=CONFIG.MASK
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=True,
        pin_memory=True,
        num_workers=4,
        drop_last=True,
    )
    
    optimizer = optim.AdamW(st_model.parameters(), lr=CONFIG.lr)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, eta_min=1e-4)
    criterion = nn.BCELoss()
    metric = RMSE()
    best_score = np.inf
    best_loss = np.inf
    
    
    for epoch in range(CONFIG.epoch):
        start_time = time.time()
        # train
        avg_loss = train_fn(train_loader, st_model, criterion, optimizer, scheduler, CONFIG.batch_size, epoch, CONFIG.device)
#        # eval
        ttac_model = tta.ClassificationTTAWrapper(st_model, tta.aliases.hflip_transform())
        avg_val_loss, preds = valid_fn(valid_loader, ttac_model, criterion, CONFIG.device)
        valid_labels = torch.tensor(valid_folds["Pawpularity"].values).float()
        score = metric(preds * 100, valid_labels)
        elapsed = time.time() - start_time
        
        LOGGER.info(
            f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  lr: {scheduler.get_lr()[0]:.8f} time: {elapsed:.0f}s"
        )
        LOGGER.info(f"Epoch {epoch+1} - Score: {score}")
        if score < best_score:
            best_score = score
            LOGGER.info(f"Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model")
            torch.save(
                {"model": st_model.state_dict(), "preds": preds * 100}, CONFIG.OUTPUT_DIR / f"{CONFIG.STUDENT_MODEL_NAME}_{fold_}_best.pth")
            
        earlystopping(avg_val_loss, st_model) #callメソッド呼び出し
        if earlystopping.early_stop: #ストップフラグがTrueの場合、breakでforループを抜ける
            print("Early Stopping!")
            break
            
    check_point = torch.load(CONFIG.OUTPUT_DIR / f"{CONFIG.STUDENT_MODEL_NAME}_{fold_}_best.pth")

    valid_folds["preds"] = check_point["preds"]

    return valid_folds

In [12]:
def get_result(result_df):
    metric = RMSE()
    preds = result_df["Pawpularity"].values
    labels = result_df["preds"].values
    score = metric(preds, labels)
    LOGGER.info(f"Score: {score:<.5f}")

In [13]:
def inference():
    predictions = []
    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    test_dataset = PawpularDataset(
        test_df, targets=None,
        augmentations=test_aug, mask=CONFIG.MASK
    )
    test_loader = DataLoader(
        test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True
    )

    for fold in range(5):
        LOGGER.info(f"========== model: {CONFIG.STUDENT_MODEL_NAME} fold: {fold} inference ==========")
        cl_model = pet_categor_extract_model(class_num=37)
        cl_model.to(CONFIG.device)
#         cl_model.load_state_dict(fix_model_state_dict(torch.load('../input/pretrained_models/efficientnet_b0_Oxford_classifier_size_224.pth')["model"]))

        model = PawpularModel(cl_model, model_name=CONFIG.STUDENT_MODEL_NAME)
        model.to(CONFIG.device)
        if torch.cuda.device_count()>1:
            model=nn.DataParallel(model)
            model.load_state_dict(torch.load(CONFIG.OUTPUT_DIR / f"{CONFIG.STUDENT_MODEL_NAME}_{fold}_best.pth")["model"])
        else:
            model.load_state_dict(fix_model_state_dict(torch.load(CONFIG.OUTPUT_DIR / f"{CONFIG.STUDENT_MODEL_NAME}_{fold}_best.pth")["model"]))
        model.eval()
        ttac_model = tta.ClassificationTTAWrapper(model, tta.aliases.hflip_transform())
        preds = []
        for i, data in tqdm(enumerate(test_loader), total=len(test_loader)):
            img,target = data['image'], data['targets']
            img = img.to(CONFIG.device)
            target = target.to(CONFIG.device)
            with torch.no_grad():
                y_preds = ttac_model(img)
            preds.append(y_preds.to("cpu").numpy())
        preds = np.concatenate(preds)
        predictions.append(preds)
    predictions = np.mean(predictions, axis=0)
    return predictions * 100

In [14]:
def main():
    # Training
    oof_df = pd.DataFrame()
    if CONFIG.training_step:
        for fold in range(CONFIG.fold):
            _oof_df = train_loop(train_df, fold)
            oof_df = pd.concat([oof_df, _oof_df])
            LOGGER.info(f"========== fold: {fold} result ==========")
            get_result(_oof_df)
        # Save OOF result
        oof_df.to_csv(CONFIG.OUTPUT_DIR / "oof_df.csv", index=False)
    else:
        oof_df = pd.read_csv(CONFIG.OUTPUT_DIR / "oof_df.csv")
    # CV result
    LOGGER.info(f"========== CV ==========")
    get_result(oof_df)
    # Inference
    predictions = inference()
    # submission
    submission = test_df.copy()
    submission["Pawpularity"] = predictions
    submission = submission[["Id", "Pawpularity"]]
    submission.to_csv(CONFIG.OUTPUT_DIR / "submission.csv", index=False)

In [15]:
if __name__ == "__main__":
    main()

========== fold: 0 training ==========
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:18<00:00,  2.50it/s]
Epoch 1 - avg_train_loss: 0.6494  avg_val_loss: 0.6437  lr: 0.00001055 time: 229s
Epoch 1 - Score: 18.02996826171875
Epoch 1 - Save Best Score: 18.0300 Model
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:19<00:00,  2.48it/s]
Epoch 2 - avg_train_loss: 0.6464  avg_val_loss: 0.6432  lr: 0.00001220 time: 231s
Epoch 2 - Score: 17.955162048339844
Epoch 2 - Save Best Score: 17.9552 Model
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:18<00:00,  2.49it/s]
Epoch 3 - avg_train_loss: 0.6461  avg_val_loss: 0.6431  lr: 0.00001490 time: 230s
Epoch 3 - Score: 17.944658279418945
Epoch 3 - Save Best Score: 17.9447 Model
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:19<00:00,  2.48it/s]
Epoch 4 - avg_train_loss: 0.6458  avg_val_loss: 0.6427  lr: 0.000

Early Stopping!


========== fold: 0 result ==========
Score: 17.89303
========== fold: 1 training ==========
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:19<00:00,  2.48it/s]
Epoch 1 - avg_train_loss: 0.6511  avg_val_loss: 0.6434  lr: 0.00001055 time: 231s
Epoch 1 - Score: 18.008718490600586
Epoch 1 - Save Best Score: 18.0087 Model
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:18<00:00,  2.50it/s]
Epoch 2 - avg_train_loss: 0.6478  avg_val_loss: 0.6432  lr: 0.00001220 time: 230s
Epoch 2 - Score: 17.983800888061523
Epoch 2 - Save Best Score: 17.9838 Model
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:18<00:00,  2.49it/s]
Epoch 3 - avg_train_loss: 0.6474  avg_val_loss: 0.6438  lr: 0.00001490 time: 230s
Epoch 3 - Score: 18.054828643798828
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:18<00:00,  2.49it/s]
Epoch 4 - avg_train_loss: 0.6473  avg_val_loss: 0.64

Early Stopping!


========== fold: 1 result ==========
Score: 17.90985
========== fold: 2 training ==========
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:18<00:00,  2.49it/s]
Epoch 1 - avg_train_loss: 0.6431  avg_val_loss: 0.6414  lr: 0.00001055 time: 230s
Epoch 1 - Score: 17.740577697753906
Epoch 1 - Save Best Score: 17.7406 Model
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:17<00:00,  2.50it/s]
Epoch 2 - avg_train_loss: 0.6367  avg_val_loss: 0.6416  lr: 0.00001220 time: 229s
Epoch 2 - Score: 17.753074645996094
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:19<00:00,  2.49it/s]
Epoch 3 - avg_train_loss: 0.6357  avg_val_loss: 0.6411  lr: 0.00001490 time: 230s
Epoch 3 - Score: 17.703947067260742
Epoch 3 - Save Best Score: 17.7039 Model
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:18<00:00,  2.49it/s]
Epoch 4 - avg_train_loss: 0.6354  avg_val_loss: 0.64

...

========== fold: 2 result ==========
Score: 17.70395
========== fold: 3 training ==========
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:18<00:00,  2.50it/s]
Epoch 1 - avg_train_loss: 0.6421  avg_val_loss: 0.6458  lr: 0.00001055 time: 230s
Epoch 1 - Score: 18.324325561523438
Epoch 1 - Save Best Score: 18.3243 Model
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:18<00:00,  2.49it/s]
Epoch 2 - avg_train_loss: 0.6376  avg_val_loss: 0.6464  lr: 0.00001220 time: 230s
Epoch 2 - Score: 18.39824676513672
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:17<00:00,  2.50it/s]
Epoch 3 - avg_train_loss: 0.6371  avg_val_loss: 0.6431  lr: 0.00001490 time: 229s
Epoch 3 - Score: 17.996339797973633
Epoch 3 - Save Best Score: 17.9963 Model
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:18<00:00,  2.49it/s]
Epoch 4 - avg_train_loss: 0.6371  avg_val_loss: 0.645

Early Stopping!


========== fold: 3 result ==========
Score: 17.98269
========== fold: 4 training ==========
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:18<00:00,  2.50it/s]
Epoch 1 - avg_train_loss: 0.6398  avg_val_loss: 0.6436  lr: 0.00001055 time: 230s
Epoch 1 - Score: 18.065101623535156
Epoch 1 - Save Best Score: 18.0651 Model
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:17<00:00,  2.50it/s]
Epoch 2 - avg_train_loss: 0.6351  avg_val_loss: 0.6423  lr: 0.00001220 time: 229s
Epoch 2 - Score: 17.903039932250977
Epoch 2 - Save Best Score: 17.9030 Model
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:17<00:00,  2.50it/s]
Epoch 3 - avg_train_loss: 0.6342  avg_val_loss: 0.6429  lr: 0.00001490 time: 229s
Epoch 3 - Score: 17.964635848999023
100%|█████████████████████████████████████████████████████████████████| 495/495 [03:17<00:00,  2.51it/s]
Epoch 4 - avg_train_loss: 0.6340  avg_val_loss: 0.64

Early Stopping!


========== fold: 4 result ==========
Score: 17.87620
========== CV ==========
Score: 17.87338
========== model: swin_large_patch4_window7_224 fold: 0 inference ==========
100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.95it/s]
========== model: swin_large_patch4_window7_224 fold: 1 inference ==========
100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.70it/s]
========== model: swin_large_patch4_window7_224 fold: 2 inference ==========
100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.92it/s]
========== model: swin_large_patch4_window7_224 fold: 3 inference ==========
100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.90it/s]
========== model: swin_large_patch4_window7_224 fold: 4 inference ==========
100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.69it